In [ ]:
""" This will create a local cached version of the requests that I can use for testing.."""
import requests
import requests_cache
##jeckhart/requests-cache   |   0.4.10 | conda           | osx-64
#conda install -c https://conda.anaconda.org/USERNAME PACKAGE

import pymongo
#requests_cache.install_cache('isic_cache')  ##Creates local sqllite database


girder_token = None
def login(username, password):
    login_url = 'https://isic-archive.com/api/v1/user/authentication'
    login_response = requests.get(login_url, auth=(username, password))
    login_response.raise_for_status()
    global girder_token
    girder_token = login_response.json()['authToken']['token']

def apiGet(endpoint, as_json=True):
    url = 'https://isic-archive.com/api/v1/%s' % endpoint
    headers = {}
    global girder_token
    if girder_token:
        headers['Girder-Token'] = girder_token
    response = requests.get(url, headers=headers)
    if as_json:
        return response.json()
    else:
        return response

def getStudies():
    """ Get a list of all studies. """
    return apiGet('study')

def getStudyDetail(study_id):
    """ Get the detailed metadata for a study. """
    return apiGet('study/%s' % study_id)

def getFeaturesetDetail(featureset_id):
    """ Get the details of a featureset. """
    return apiGet('featureset/%s' % featureset_id)

def getStudyUsers(study_id):
    """ Get a list of users in a study. """
    return apiGet('study/%s/users' % study_id)

def getStudyImages(study_id):
    """ Get a list of images in a study. """
    return apiGet('study/%s/images' % study_id)

def getImageDetail(image_id):
    """ Get the detailed metadata for an image. """
    return apiGet('image/%s' % image_id)

def getImageFile(image_id):
    """ Get the original JPEG file for an image. """
    return apiGet('image/%s/download' % image_id, as_json=False)

def getImageThumbnail(image_id, width=None):
    """ Get the thumbnail-size JPEG preview for an image.
        Width defaults to 256 if not specified. """
    url = 'image/%s/thumbnail' % image_id
    if width is not None:
        url += '?width=%s' % width
    return apiGet(url, as_json=False)

def getAnnotations(study_id, user_id=None, image_id=None):
    """ Get the list of annotations for a study (optionally filtering by user or image). """
    url = 'annotation?studyId=' + study_id
    if user_id:
        url += '&userId=' + user_id
    if image_id:
        url += '&imageId=' + image_id
    return apiGet(url)

def getAnnotationDetail(annotation_id):
    """ Get the details of an annotation, including the feature values. """
    return apiGet('annotation/%s' % annotation_id)

def getSegmentationDetail(segmentation_id):
    """ Get the details of a segmentation, including the lesion boundary. """
    return apiGet('segmentation/%s' % segmentation_id)

def getSegmentationSuperpixels(segmentation_id):
    """ Get the PNG-encoded superpixels file for a segmentation. """
    return apiGet('segmentation/%s/superpixels' % segmentation_id, as_json=False)

def getSegmentationThumbnail(segmentation_id, width=None):
    """ Get the a thumbnail-size JPEG preview of the lesion boundary for a segmentation.
    Width defaults to 256 if not specified. """
    url = 'segmentation/%s/thumbnail' % segmentation_id
    if width is not None:
        url += '?width=%s' % width
    return apiGet(url, as_json=False)

In [ ]:
""" Enter your credentials here, but DO NOT save and commit this Notebook with them present. """
username = ''
password = ''
login(username, password)

In [ ]:
study_list = getStudies()
for study in study_list:
    study_featureset = getFeaturesetDetail(study['meta']['featuresetId'])
    study_images = getStudyImages(study['_id'])
    study_users = getStudyUsers(study['_id'])
    
    print 'Study: %s (%s)' % (study['name'], study['_id'])
    print '  Featureset: %s' % study_featureset['name']
    print '  Images: %s' % len(study_images)
    print '  Users: %s' % len(study_users)

In [ ]:
study = getStudyDetail('55c4668a9fc3c1536a0130c8')  # UDA2_pilot
study_images = getStudyImages(study['_id'])
study_users = getStudyUsers(study['_id'])

print study['name']
for image in study_images:
    print ' ', image['name']
    # Fetch the details, so we can get metadata
    image = getImageDetail(image['_id'])
    for user in study_users:
        print '   ', user['login'],
        annotations = getAnnotations(
            study_id=study['_id'],
            image_id=image['_id'],
            user_id=user['_id']
        )
        # A study + image + user should fully define a single annotation
        assert len(annotations) == 1
        
        # We need to individually fetch annotations in order to get the feature values
        annotation = getAnnotationDetail(annotations[0]['_id'])
        
        if annotation.get('stopTime'):
            print 'complete'
        else:
            print 'pending'
            continue
        
        if annotation.get('segmentationId'):
            segmentation = getSegmentationDetail(annotation['segmentationId'])
        else:
            # Some old annotations don't have the segmentation ID attached yet
            # So, grab the first (most recent) segmentation associated with the image
            segmentation = apiGet('segmentation?imageId=%s' % annotation['imageId'])[0]

        image_metadata = image['meta']
        superpixels = getSegmentationSuperpixels(segmentation['_id'])
        feature_values = annotation['annotations']
        # There is some inconsistancy in how the annotations are currently stored
        if 'region_features' in feature_values:
            feature_values = feature_values['region_features']

        # TODO: do something with the image_metadata, superpixels, and feature_values
